## _*Using Qiskit Aqua for Maxcut Problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE quantum algorithm to compute the max cut of a given graph. 

The problem is defined as follows. Given a graph $G = (V,E)$ with weights $w_{ij}$ on the edges, we are looking for a subset $S \subseteq V$ such that $\sum_{(i,j) \in E : i \in S, j \not\in S} w_{ij}$ is maximized.

The graph provided as an input is used first to generate an Ising Hamiltonian, which is then passed as an input to VQE.  As a reference, this notebook also computes the max cut using the Exact Eigensolver classical algorithm.

In [1]:
import numpy as np

from qiskit.aqua import Operator
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.aqua.translators.ising import maxcut
from qiskit import Aer

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian translated from the max cut problem. We load a small sample instance of the maxcut problem.

In [2]:
w = maxcut.parse_gset_format('sample.maxcut')
qubitOp, offset = maxcut.get_maxcut_qubitops(w)

We also offer a function to generate a random graph as a input.

In [3]:
if True:
    np.random.seed(8123179)
    n = 4 # number of nodes in the graph
    w = maxcut.random_graph(n, edge_prob=0.5, weight_range=10)
    qubitOp, offset = maxcut.get_maxcut_qubitops(w)
print(w)

[[ 0.  8. -9.  0.]
 [ 8.  0.  7.  9.]
 [-9.  7.  0. -8.]
 [ 0.  9. -8.  0.]]


We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. Now the complete params can be passed to the algorithm and run. The result is a dictionary.

In [4]:
ee = ExactEigensolver(qubitOp)
result = ee.run()
# print('objective function:', maxcut.maxcut_obj(result, offset))
x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))

energy: -20.5
maxcut objective: -24.0
solution: [1. 0. 1. 1.]


Now we want VQE and so change it and add its other configuration parameters. VQE also needs and optimizer and variational form. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

In [5]:
l_bfgs_b = L_BFGS_B()
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, l_bfgs_b, 'grouped_paulis', batch_mode=True)
backend = Aer.get_backend('statevector_simulator')

result = vqe.run(backend)

x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))

energy: -20.499999999992276
time: 25.959708213806152
maxcut objective: -23.999999999992276
solution: [0. 1. 0. 0.]
